In [5]:
from pymilvus import MilvusClient, DataType
from dotenv import load_dotenv
import os

load_dotenv()
# https://milvus.io/docs/quickstart.md

client = MilvusClient(
    uri=os.getenv("MILVUS_URI"),
    user=os.getenv("MILVUS_USER"),
    password=os.getenv("MILVUS_PWD"),
)

In [6]:
# create schema
schema = client.create_schema(
    auto_id=True,
    enbale_dynamic_fields=True,
)

schema.add_field(
    field_name="pk",
    datatype=DataType.VARCHAR,
    is_primary=True,
    max_length=100,
)
schema.add_field(
    field_name="dense_vector",
    datatype=DataType.FLOAT_VECTOR,
    dim=4,
)

# set index param for vector field
index_params = client.prepare_index_params()
index_params.add_index(
    field_name="dense_vector",
    index_name="dense_vector_index",
    index_type="AUTOINDEX",
    metric_type="IP",
)

# create collection
client.create_collection(
    collection_name="my_collection_for_dense_vector",
    schema=schema,
    index_params=index_params,
)

# insert data
data = [
    {"dense_vector": [0.1, 0.2, 0.3, 0.7]},
    {"dense_vector": [0.2, 0.3, 0.4, 0.8]},
]
client.insert(
    collection_name="my_collection_for_dense_vector",
    data=data,
)

{'insert_count': 2, 'ids': ['458312171705069301', '458312171705069302'], 'cost': 1}

In [7]:
# perform similarity search
search_params = {
    "params": {"nprobe": 10}
}
query_vector = [0.1, 0.2, 0.3, 0.7]

res = client.search(
    collection_name="my_collection_for_dense_vector",
    data=[query_vector],
    anns_field="dense_vector",
    search_params=search_params,
    limit=5,
    output_fields=["pk"],
)

print(res)

data: [[{'pk': '458312171705069302', 'distance': 0.7599999904632568, 'entity': {'pk': '458312171705069302'}}, {'pk': '458312171705069301', 'distance': 0.6299999952316284, 'entity': {'pk': '458312171705069301'}}]],{'cost': 6}
